In [1]:
import itertools as it
import os
import pickle
import re
from copy import deepcopy
from functools import reduce
from glob import glob
from operator import add, itemgetter
from pprint import pprint

import gensim
import gensim.downloader as api
import matplotlib
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from scipy.spatial.distance import pdist, squareform
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import nan_euclidean_distances
from sklearn.preprocessing import LabelEncoder, PowerTransformer
from sklearn.decomposition import PCA
import tqdm

from utils import video_id_without_categories

/home/hylomorph/Study/ped/yt_trending_videos/venv/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
df = pd.read_pickle('data/chkp3.pkl')
mask = pd.read_pickle('data/known_unknown_mask.pkl')

#### Wczytywanie danych

In [4]:
known_df = df[mask]
print(known_df.shape)
known_df.head(3)

(2838, 249)


,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,publish_time_day,publish_time_month,publish_time_year,...,objects_5,objects_6,objects_7,objects_8,objects_9,objects_10,objects_11,objects_12,objects_13,objects_14
video_id,,,,,,,,,,,,,,,,,,,,,
wnwF1FHybDQ,225286,1731,193,206,0,0,0,10,11,2017,...,-0.007956,0.000595,-0.001807,-0.000473,-0.001074,0.001726,0.000417,-0.000837,-0.000862,-0.000338
fIQ2Ty2OL34,18573,420,10,236,0,0,0,10,11,2017,...,0.000833,0.007008,-0.004601,-0.004798,-0.003009,-0.005794,-0.000954,-0.004876,-0.005280,-0.000027
1Wk8ZRgXQnY,41251,1561,29,96,0,0,0,9,11,2017,...,0.000833,0.007008,-0.004601,-0.004798,-0.003009,-0.005794,-0.000954,-0.004876,-0.005280,-0.000027


In [5]:
uknown_df = df[~mask]
print(uknown_df.shape)
uknown_df.head(3)

(5768, 249)


,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,publish_time_day,publish_time_month,publish_time_year,...,objects_5,objects_6,objects_7,objects_8,objects_9,objects_10,objects_11,objects_12,objects_13,objects_14
video_id,,,,,,,,,,,,,,,,,,,,,
9wRQljFNDW8,81377,655,25,177,0,0,0,13,11,2017,...,-0.007988,0.000606,-0.001781,-0.000445,-0.001092,0.001665,0.000452,-0.000808,-0.000873,-0.000329
Om_zGhJLZ5U,288922,7515,792,2111,0,0,0,12,11,2017,...,-0.007947,0.000590,-0.001807,-0.000466,-0.001071,0.001721,0.000426,-0.000836,-0.000861,-0.000348
goP4Z5wyOlM,34785,308,26,413,0,0,0,12,11,2017,...,0.000844,0.007013,-0.004608,-0.004800,-0.003007,-0.005776,-0.000960,-0.004892,-0.005275,-0.000043


#### Czy zbiory są rozłączne

In [9]:
set(known_df.index).intersection(set(uknown_df.index))

set()